In [1]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Separate file for ID3 implementation

## Imports

In [2]:
import numpy as np
from collections import Counter

## ID3 Implementation
With guidance from: https://www.geeksforgeeks.org/iterative-dichotomiser-3-id3-algorithm-from-scratch/ 

First, we need to create a class for the node of the tree. 

In [3]:
class Node:
        def __init__(self, feature=None, value=None, results=None, true_branch=None, false_branch=None):
                self.feature = feature 
                self.value = value
                self.results = results
                self.true_branch = true_branch
                self.false_branch = false_branch

Next, we need helper function for calculating entropy.

Entropy measures the uncertainty or impurity within a set of examples, helping to determine how well an attribute can split the data. 

In [4]:
def entropy(data):
        results = Counter(data) # count the occurrences of each result
        f_entropy = 0.0 # initialize "final" entropy
        for result in results.keys():
                p = float(results[result])/len(data)
                f_entropy -= p*np.log2(p)
        return f_entropy

Now we need a function for splitting the data based on a given feature and value.

This function will return two sets of data: one where the feature is less than or equal to the value, and one where the feature is greater than the value.

X - the data
y - the labels aka target values

In [5]:
def split_data(X, y, feature, value):
        true_indices = np.where(X[:, feature] <= value)[0]
        false_indices = np.where(X[:, feature] > value)[0]
        
        true_X, true_y = X[true_indices], y[true_indices]
        false_X, false_y = X[false_indices], y[false_indices]
        
        return true_X, true_y, false_X, false_y

Here is the main function for building the tree. 

It will recursively build the tree by going through all possible values of all features and choosing the one that has the best information gain. Building the tree will stop when the information gain is 0 or when the prediction of the node is clear. 

Source: https://www.geeksforgeeks.org/iterative-dichotomiser-3-id3-algorithm-from-scratch/
with added comments for better understanding.

In [6]:
def build_tree(X, y):   # X - data, y - labels (y needs to be 1D array)
        if len(set(y)) == 1:    # if all the labels are the same, the node is a leaf
                return Node(results=y[0])

        best_gain = 0
        best_criteria = None
        best_sets = None
        n_features = X.shape[1]

        current_entropy = entropy(y)
        
        # the loop where we go through all possible values of all features
        for feature in range(n_features):
                feature_values = set(X[:, feature])
                for value in feature_values:    # go through all possible values of the feature
                        true_X, true_y, false_X, false_y = split_data(X, y, feature, value)
                        
                        # calculating the information gain of the split
                        true_entropy = entropy(true_y)
                        false_entropy = entropy(false_y)
                        p = len(true_y) / len(y)
                        gain = current_entropy - p * true_entropy - (1 - p) * false_entropy
                        
                        # if best -> store the values
                        if gain > best_gain:
                                best_gain = gain
                                best_criteria = (feature, value)
                                best_sets = (true_X, true_y, false_X, false_y)
        
        # create the next nodes after picking the best possible split
        if best_gain > 0:
                true_branch = build_tree(best_sets[0], best_sets[1])
                false_branch = build_tree(best_sets[2], best_sets[3])
                return Node(feature=best_criteria[0], value=best_criteria[1], true_branch=true_branch, false_branch=false_branch)
        
        # if the best gain is 0, the node is a leaf
        return Node(results=y[0])

This function will return the prediction of the tree. It will simply traverse the tree until it reaches a leaf node and return the result of that node, i.e., prediction of the tree.

In [7]:
def predict(tree, sample):
        if tree.results is not None:
                return tree.results
        else:
                branch = tree.false_branch
                if sample[tree.feature] <= tree.value:
                        branch = tree.true_branch
                return predict(branch, sample)

All the functions we need are now implemented. We will continue the testing in merge file.